In [8]:
import os
import sys

import time
import copy
import numpy as np
import math

sys.argv = ['options', '--classes', '40', '--dataroot', '/workspace/ModelNet/modelnet40-normal_numpy/',
           '--checkpoints_dir', '/workspace/ModelNet/SO-Net/SO-Net/checkpoints/model_1/']
from modelnet.options import Options
opt = Options().parse()  # set CUDA_VISIBLE_DEVICES before import torch

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np

from models.classifier import Model
from data.modelnet_shrec_loader import ModelNet_Shrec_Loader
from util.visualizer import Visualizer

------------ Options -------------
activation: relu
batch_size: 8
bn_momentum: 0.1
bn_momentum_decay: 0.6
bn_momentum_decay_step: None
checkpoints_dir: /workspace/ModelNet/SO-Net/SO-Net/checkpoints/model_1/
classes: 40
dataroot: /workspace/ModelNet/modelnet40-normal_numpy/
dataset: modelnet
device: cuda:0
display_id: 200
display_winsize: 256
dropout: 0.7
feature_num: 1024
gpu_id: 0
input_pc_num: 5000
k: 3
lr: 0.001
nThreads: 8
name: train
node_num: 64
normalization: batch
pretrain: None
pretrain_lr_ratio: 1
random_pc_dropout_lower_limit: 1
rot_horizontal: False
rot_perturbation: False
som_k: 9
som_k_type: avg
surface_normal: True
translation_perturbation: False
-------------- End ----------------


In [9]:
testset = ModelNet_Shrec_Loader(opt.dataroot, 'test', opt)
testloader = torch.utils.data.DataLoader(testset, batch_size=opt.batch_size, shuffle=False, num_workers=opt.nThreads)

# create model, optionally load pre-trained model
model = Model(opt)

In [17]:
checkpoints = [x for x in os.listdir(opt.checkpoints_dir) if os.path.splitext(x)[-1] == '.pth']
checkpoints = sorted(checkpoints, key=lambda x: int(x.split('_')[0]))[-2:]
for item in checkpoints:
    if item.split('_')[-1] == 'classifier.pth':
        ckp_classifier = item
    if item.split('_')[-1] == 'encoder.pth':
        ckp_encoder = item
classifier_ckp_path = os.path.join(opt.checkpoints_dir, ckp_classifier)
encoder_ckp_path = os.path.join(opt.checkpoints_dir, ckp_encoder)
model.encoder.load_state_dict(torch.load(encoder_ckp_path))
model.classifier.load_state_dict(torch.load(classifier_ckp_path))

In [53]:
model.test_loss.data.zero_()
model.test_accuracy.data.zero_()
model_probabilities = []
gt_labels = []
for i, data in enumerate(testloader):
    input_pc, input_sn, input_label, input_node, input_node_knn_I = data
    model.set_input(input_pc, input_sn, input_label, input_node, input_node_knn_I)
    model.test_model()
    model_probabilities.append(model.score.data)
    gt_labels.append(model.input_label.cpu())

model_probabilities = torch.cat(model_probabilities, 0)
gt_labels = torch.cat(gt_labels, 0)

In [65]:
predictions = torch.argmax(model_probabilities, 1).cpu()
correct_mask = torch.eq(predictions, gt_labels).float()
test_accuracy = torch.mean(correct_mask).item()
print('Test accuracy =', test_accuracy)

Test accuracy = 0.9262560606002808
